## Merging the NBA datasets

Reading in the three data frames

In [28]:
import pandas as pd
champs=pd.read_pickle("https://github.com/Policy-by-the-Numbers/spacejam/raw/main/nbachamps.pkl")
streak=pd.read_pickle("https://github.com/Policy-by-the-Numbers/spacejam/raw/main/nbawinstreaks.pkl")
mvp=pd.read_pickle("https://github.com/Policy-by-the-Numbers/spacejam/raw/main/nba_mvps.pkl")
regseason=pd.read_pickle("https://github.com/Policy-by-the-Numbers/spacejam/raw/main/regseason.pkl")

In [29]:
champs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Team      33 non-null     object 
 1   Win       27 non-null     float64
 2   Loss      27 non-null     float64
 3   Apps      27 non-null     float64
 4   ChampPct  27 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.4+ KB


In [30]:
streak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Games   33 non-null     int64 
 1   Team    33 non-null     object
dtypes: int64(1), object(1)
memory usage: 656.0+ bytes


Realizing that the unit of analysis for win streak is not by team. Gonna move on with just the MVP data to see what happens.

In [31]:
champs.shape,streak.shape,mvp.shape

((33, 5), (33, 2), (30, 2))

Looking first at championship data vs. MVP data

In [32]:
onlyChamps=set(champs.Team)-set(mvp.Team)
onlyChamps

{'Atlanta Hawks[v]',
 'Baltimore Bullets (original) (folded in 1954)[viii]',
 'Brooklyn Nets[x]',
 'Chicago Stags (folded in 1950)',
 'Detroit Pistons[iv]',
 'Golden State Warriors[ii]',
 'Los Angeles Clippers',
 'Los Angeles Lakers[i]',
 'Oklahoma City Thunder[vii]',
 'Philadelphia 76ers[iii]',
 'Sacramento Kings[ix]',
 'Washington Capitols (folded in 1951)',
 'Washington Wizards[vi]'}

In [33]:
onlyMVP=set(mvp.Team)-set(champs.Team)
onlyMVP

{'Baltimore Bullets (now Washington Wizards)',
 'Brooklyn Nets',
 'Buffalo Braves (now Los Angeles Clippers)',
 'Cincinnati Royals (now Sacramento Kings)',
 'Detroit Pistons',
 'Los Angeles Lakers',
 'Oklahoma City Thunder',
 'Philadelphia 76ers',
 'Philadelphia/Golden State Warriors',
 'St. Louis Hawks (now Atlanta Hawks)'}

Here we'll try and find what countries in onlyChamps may match those in onlyMVP using fuzzy merge

In [34]:
!pip install thefuzz
from thefuzz import process as fz

# look for a country in onlyMVP and return the most similar
[(fz.extractOne(champs, onlyMVP),champs) for champs in sorted(onlyChamps)]

[(('St. Louis Hawks (now Atlanta Hawks)', 86), 'Atlanta Hawks[v]'),
 (('Baltimore Bullets (now Washington Wizards)', 57),
  'Baltimore Bullets (original) (folded in 1954)[viii]'),
 (('Brooklyn Nets', 95), 'Brooklyn Nets[x]'),
 (('Detroit Pistons', 44), 'Chicago Stags (folded in 1950)'),
 (('Detroit Pistons', 95), 'Detroit Pistons[iv]'),
 (('Philadelphia/Golden State Warriors', 88), 'Golden State Warriors[ii]'),
 (('Buffalo Braves (now Los Angeles Clippers)', 90), 'Los Angeles Clippers'),
 (('Los Angeles Lakers', 95), 'Los Angeles Lakers[i]'),
 (('Oklahoma City Thunder', 95), 'Oklahoma City Thunder[vii]'),
 (('Philadelphia 76ers', 95), 'Philadelphia 76ers[iii]'),
 (('Cincinnati Royals (now Sacramento Kings)', 86), 'Sacramento Kings[ix]'),
 (('Baltimore Bullets (now Washington Wizards)', 44),
  'Washington Capitols (folded in 1951)'),
 (('Baltimore Bullets (now Washington Wizards)', 86),
  'Washington Wizards[vi]')]

In [35]:
[(fz.extractOne(champs, onlyMVP),champs)
 for champs in sorted(onlyChamps)
if fz.extractOne(champs, onlyMVP)[1]>80]

[(('St. Louis Hawks (now Atlanta Hawks)', 86), 'Atlanta Hawks[v]'),
 (('Brooklyn Nets', 95), 'Brooklyn Nets[x]'),
 (('Detroit Pistons', 95), 'Detroit Pistons[iv]'),
 (('Philadelphia/Golden State Warriors', 88), 'Golden State Warriors[ii]'),
 (('Buffalo Braves (now Los Angeles Clippers)', 90), 'Los Angeles Clippers'),
 (('Los Angeles Lakers', 95), 'Los Angeles Lakers[i]'),
 (('Oklahoma City Thunder', 95), 'Oklahoma City Thunder[vii]'),
 (('Philadelphia 76ers', 95), 'Philadelphia 76ers[iii]'),
 (('Cincinnati Royals (now Sacramento Kings)', 86), 'Sacramento Kings[ix]'),
 (('Baltimore Bullets (now Washington Wizards)', 86),
  'Washington Wizards[vi]')]

I like these matches, so I'll create a dictionary:

In [36]:
changesMVP1={fz.extractOne(champs, onlyMVP)[0]:champs
            for champs in sorted(onlyChamps)
            if fz.extractOne(champs, onlyMVP)[1]>80}
              
# dictionary of matches
changesMVP1

{'St. Louis Hawks (now Atlanta Hawks)': 'Atlanta Hawks[v]',
 'Brooklyn Nets': 'Brooklyn Nets[x]',
 'Detroit Pistons': 'Detroit Pistons[iv]',
 'Philadelphia/Golden State Warriors': 'Golden State Warriors[ii]',
 'Buffalo Braves (now Los Angeles Clippers)': 'Los Angeles Clippers',
 'Los Angeles Lakers': 'Los Angeles Lakers[i]',
 'Oklahoma City Thunder': 'Oklahoma City Thunder[vii]',
 'Philadelphia 76ers': 'Philadelphia 76ers[iii]',
 'Cincinnati Royals (now Sacramento Kings)': 'Sacramento Kings[ix]',
 'Baltimore Bullets (now Washington Wizards)': 'Washington Wizards[vi]'}

In [37]:
mvp.Team.replace(to_replace=changesMVP1,inplace=True)
mvp.Team

0                 Boston Celtics
1          Los Angeles Lakers[i]
2        Philadelphia 76ers[iii]
3                  Chicago Bulls
4                Milwaukee Bucks
5                Houston Rockets
6              San Antonio Spurs
7                   Phoenix Suns
8      Golden State Warriors[ii]
9               Atlanta Hawks[v]
10                     Utah Jazz
11           Cleveland Cavaliers
12                    Miami Heat
13    Oklahoma City Thunder[vii]
14                Denver Nuggets
15          Sacramento Kings[ix]
16        Washington Wizards[vi]
17               New York Knicks
18          Los Angeles Clippers
19        Portland Trail Blazers
20        Minnesota Timberwolves
21              Dallas Mavericks
22              Brooklyn Nets[x]
23             Charlotte Hornets
24           Detroit Pistons[iv]
25                Indiana Pacers
26             Memphis Grizzlies
27          New Orleans Pelicans
28                 Orlando Magic
29               Toronto Raptors
Name: Team

In [38]:
# second try
onlyChamps=set(champs.Team)-set(mvp.Team)
onlyMVP=set(mvp.Team)-set(champs.Team)
[(fz.extractOne(champs, onlyMVP),champs) for champs in sorted(onlyChamps)]

[(None, 'Baltimore Bullets (original) (folded in 1954)[viii]'),
 (None, 'Chicago Stags (folded in 1950)'),
 (None, 'Washington Capitols (folded in 1951)')]

The remaining teams are ones that are no longer existent and therefore are likely not in the list of MVPs by team. Let's retry the merge.

In [39]:
champs.merge(mvp).shape #much better matching

(30, 6)

In [40]:
champs_mvp=champs.merge(mvp)
champs_mvp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Team      30 non-null     object 
 1   Win       24 non-null     float64
 2   Loss      24 non-null     float64
 3   Apps      24 non-null     float64
 4   ChampPct  24 non-null     float64
 5   Awards    30 non-null     int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.6+ KB


## Here I'm going to try and merge the regular season data

In [41]:
champs_mvp.merge(regseason).shape

(21, 11)

We lose nine teams. Let's see if we can recover any via fuzzy merge.

In [43]:
OnlyChampMVP=set(champs_mvp.Team)-set(regseason.Team)
OnlyChampMVP

{'Atlanta Hawks[v]',
 'Brooklyn Nets[x]',
 'Detroit Pistons[iv]',
 'Golden State Warriors[ii]',
 'Los Angeles Lakers[i]',
 'Oklahoma City Thunder[vii]',
 'Philadelphia 76ers[iii]',
 'Sacramento Kings[ix]',
 'Washington Wizards[vi]'}

In [44]:
OnlyReg=set(regseason.Team)-set(champs_mvp.Team)
OnlyReg

{'Atlanta Hawks',
 'Brooklyn Nets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Los Angeles Lakers',
 'Oklahoma City Thunder',
 'Philadelphia 76ers',
 'Sacramento Kings',
 'Washington Wizards'}

Doing the Fuzzy Merge

In [47]:
from thefuzz import process as fz

# look for a country in onlyReg and return the most similar
[(fz.extractOne(champs_mvp, OnlyReg),champs_mvp) for champs_mvp in sorted(OnlyChampMVP)]

[(('Atlanta Hawks', 95), 'Atlanta Hawks[v]'),
 (('Brooklyn Nets', 95), 'Brooklyn Nets[x]'),
 (('Detroit Pistons', 95), 'Detroit Pistons[iv]'),
 (('Golden State Warriors', 95), 'Golden State Warriors[ii]'),
 (('Los Angeles Lakers', 95), 'Los Angeles Lakers[i]'),
 (('Oklahoma City Thunder', 95), 'Oklahoma City Thunder[vii]'),
 (('Philadelphia 76ers', 95), 'Philadelphia 76ers[iii]'),
 (('Sacramento Kings', 95), 'Sacramento Kings[ix]'),
 (('Washington Wizards', 95), 'Washington Wizards[vi]')]

Keeping those with >=95% matches

In [53]:
changesReg1={fz.extractOne(champs_mvp, OnlyReg)[0]:champs_mvp
            for champs_mvp in sorted(OnlyChampMVP)
            if fz.extractOne(champs_mvp, OnlyReg)[1]>94}
changesReg1

{'Atlanta Hawks': 'Atlanta Hawks[v]',
 'Brooklyn Nets': 'Brooklyn Nets[x]',
 'Detroit Pistons': 'Detroit Pistons[iv]',
 'Golden State Warriors': 'Golden State Warriors[ii]',
 'Los Angeles Lakers': 'Los Angeles Lakers[i]',
 'Oklahoma City Thunder': 'Oklahoma City Thunder[vii]',
 'Philadelphia 76ers': 'Philadelphia 76ers[iii]',
 'Sacramento Kings': 'Sacramento Kings[ix]',
 'Washington Wizards': 'Washington Wizards[vi]'}

In [54]:
regseason.Team.replace(to_replace=changesReg1,inplace=True)

In [56]:
champs_mvp.merge(regseason).shape #30 teams! nice!

(30, 11)

In [57]:
champs_mvp_reg=champs_mvp.merge(regseason)

In [58]:
champs_mvp_reg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Team          30 non-null     object  
 1   Win           24 non-null     float64 
 2   Loss          24 non-null     float64 
 3   Apps          24 non-null     float64 
 4   ChampPct      24 non-null     float64 
 5   Awards        30 non-null     int64   
 6   GP2022        30 non-null     int64   
 7   TotalWins     30 non-null     int64   
 8   TotalLosses   30 non-null     int64   
 9   RegSeasonPct  30 non-null     float64 
 10  Division      30 non-null     category
dtypes: category(1), float64(5), int64(4), object(1)
memory usage: 2.8+ KB


Data Scaling

In [59]:
champs_mvp_reg.describe(include='all')

,Team,Win,Loss,Apps,ChampPct,Awards,GP2022,TotalWins,TotalLosses,RegSeasonPct,Division
count,30,24.000000,24.000000,24.000000,24.000000,30.000000,30.00000,30.000000,30.000000,30.00000,30
unique,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
top,New York Knicks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Southwest
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
mean,NaN,3.125000,3.083333,6.208333,0.424250,2.233333,4167.40000,2088.266667,2079.133333,0.49570,NaN
std,NaN,4.665437,3.091597,7.113852,0.328651,2.528231,1308.42555,739.716578,636.538578,0.05147,NaN
min,NaN,0.000000,0.000000,1.000000,0.000000,0.000000,1604.00000,740.000000,864.000000,0.39900,NaN
25%,NaN,1.000000,1.000000,2.000000,0.237500,0.250000,3460.00000,1610.000000,1603.250000,0.46100,NaN
50%,NaN,1.000000,2.500000,4.000000,0.381000,1.500000,4197.00000,2230.000000,2074.000000,0.49600,NaN
75%,NaN,3.000000,4.000000,6.250000,0.604000,3.000000,5560.50000,2583.750000,2445.250000,0.52525,NaN


Let's pay attention to highest and lowest values

In [60]:
champs_mvp_reg.describe().loc[['min','max']].T

,min,max
Win,0.000,17.000
Loss,0.000,15.000
Apps,1.000,32.000
ChampPct,0.000,1.000
Awards,0.000,10.000
GP2022,1604.000,5950.000
TotalWins,740.000,3513.000
TotalLosses,864.000,3187.000
RegSeasonPct,0.399,0.611


Now I think we're ready to export

## Exporting merged dataset

For future use in Python:

In [61]:
champs_mvp_reg.to_pickle("champs_mvp_reg.pkl")
# you will need: DF=pd.read_pickle("champs_mvp.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://..../champs_mvp.pkl"),compression=None)

For future use in R:

In [62]:
#try the following before starting Python:
#export LD_LIBRARY_PATH="$(python -m rpy2.situation LD_LIBRARY_PATH)":${LD_LIBRARY_PATH}

!pip install rpy2.robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(champs_mvp_reg,file="champs_mvp_reg.RDS")


#In R, you call it with: DF = readRDS("demo_fragile.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://..../demo_fragile.RDS")

ERROR: Could not find a version that satisfies the requirement rpy2.robjects (from versions: none)
ERROR: No matching distribution found for rpy2.robjects


ModuleNotFoundError: No module named 'rpy2'

In [ ]:
Gonna save as a csv also just cause:

In [63]:
champs_mvp.to_csv("champs_mvp_reg.csv")